<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# ![Spark Logo Tiny](https://www.quantiaconsulting.com/logos/logo_spark_tiny.png) Reading Data Lab
* The goal of this lab is to put into practice some of what you have learned about reading data with Apache Spark python pandas.

## Instructions
0. Start with the file **data/data_geo.csv**.
0. Inspect the content of the file and ask yourself:
    * What is the separator?
    * Is there an header?
    ...
0. Read in the file using pyspark in two different ways
    1 - Let system infer the schema
    2 - Manually pass the schema
0. Use pyspark to save the dataframe as a `parquet` in the `/home/jovyan/data/pyspark` folder
0. Use pyspark to save the dataframe as a `table` in the default db
0. Perform some exploration queries:
    * Extract the `2015 Median Home Prices`
    * Show the top 10 cities by `2015 Median Sales Price`
    * Show the top 10 cities with the `2015 Median Sales Price` >= $ 300,000
    * ...

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

import os
import qcutils
from pyspark.sql import SparkSession
import boto3
import io
import pandas
import s3fs

s3 = boto3.client('s3')
psBaseUri = "s3a://quantia-master/training/"

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.5 pyspark-shell'

spark = (SparkSession.builder 
    .master("local[*]")
    .appName("test")
    .getOrCreate()
        )
qcutils.init_spark_session(spark)

spark

## Explore data

You can use the `head data/data_geo.csv` in the terminal.

You get

```
2014 rank,City,State,State Code,2014 Population estimate,2015 median sales price
101,Birmingham,Alabama,AL,212247,162.9
125,Huntsville,Alabama,AL,188226,157.7
122,Mobile,Alabama,AL,194675,122.5
114,Montgomery,Alabama,AL,200481,129
64,Anchorage[19],Alaska,AK,301010,null
78,Chandler,Arizona,AZ,254276,null
86,Gilbert[20],Arizona,AZ,239277,null
88,Glendale,Arizona,AZ,237517,null
38,Mesa,Arizona,AZ,464704,null
```

`data_geo.csv` is a common csv file with a `,` as a separator and qith the header (first line before the `\n` character).

## Read Data

### PySpark with inferSchema

In [ ]:
psCsvPath = psBaseUri + "data_geo.csv"

psTestDF = (spark.read
            .option("header", True)
            .option("inferschema", True)
            .csv(psCsvPath)
           )

psTestDF.printSchema()

### PySpark with user-defined Schema

In [ ]:
from pyspark.sql.types import *

psCsvPath = psBaseUri + "data_geo.csv"

csvSchema = StructType([
  StructField("2014_rank", IntegerType(), nullable=False),
  StructField("City", StringType(), nullable=False),
  StructField("State", StringType(), nullable=False),
  StructField("State_Code", StringType(), nullable=False),
  StructField("2014_Population_estimate", StringType(), nullable=False),
  StructField("2015_Median_sales_price", StringType(), nullable=True),
])

psTestDF = (spark.read
            .option("header", True)
            .option("inferschema", True)
            .schema(csvSchema)
            .csv(psCsvPath)
           )

psTestDF.printSchema()

In [ ]:
psTestDF

***NOTE*** `null` are of type `String`!!!

In [ ]:
psTestDF.filter(psTestDF["2015_Median_sales_price"] == "null")

In [ ]:
psTestDF.filter(psTestDF["2015_Median_sales_price"] == "null").count()

we need to use `withColumn` e `cast` to make them **real nulls**

In [ ]:
castedPsTestDF = (psTestDF.withColumn("2015_Median_sales_price",psTestDF["2015_Median_sales_price"].cast("float"))
            .withColumn("2014_Population_estimate",psTestDF["2014_Population_estimate"].cast("float"))
           )

In [ ]:
castedPsTestDF

In [ ]:
castedPsTestDF.filter(castedPsTestDF["2015_Median_sales_price"] == "null")

In [ ]:
castedPsTestDF.filter(castedPsTestDF["2015_Median_sales_price"] == "null").count()

## Write Data

## Parquet

In [ ]:
castedPsTestDF.write.parquet("/home/jovyan/data/pyspark/data_geo.parquet")

## Table

NOTA: usare overwrite per poter sovrascrivere una tabella già salvata

In [ ]:
castedPsTestDF.write.mode("overwrite").saveAsTable("data_geo")

## Explore Data

### Extract the `2015 Median Home Prices`

In [ ]:
spark.sql("""
SELECT State_Code, 2015_median_sales_price 
FROM data_geo
""")

### Extract the top 10 cities by `2015 Median Sales Price`

In [ ]:
spark.sql("""
SELECT
    City, 
    2014_Population_estimate/1000 AS 2014_Population_Estimate_1000, 
    2015_median_sales_price AS 2015_Median_Sales_Price_1000
FROM data_geo 
ORDER BY 2015_median_sales_price DESC
LIMIT 10
""")

### Extract the top 10 cities with the `2015 Median Sales Price` >= $ 300,000

In [ ]:
spark.sql("""
SELECT
    City, 
    State_Code, 
    2015_median_sales_price
FROM data_geo 
WHERE 2015_median_sales_price >= 300
ORDER BY 2015_median_sales_price DESC
LIMIT 10
""")